# Data Engineer Instructions

## Introducción

  Houm es una startup que permite administrar, arrendar y vender propiedades rápido, seguro y fácil a miles de usuarios en Latinoamérica. En su desafío por entregar la mejor experiencia de usuarios, la empresa y su head de operaciones le pide hacer un estudio sobre el comportamiento de sus visitas en torno al clima. Es por esto que lo contactan a usted para dar respuestas a las preguntas de la compañía.


  Carlos, nuestro data scientist, le informa que existen tres fuentes de información relevantes, las cuales contienen información de la operación que podrían ser de utilidad. Los tres archivos se detallan a continuación (presentes en el archivo dataset.zip adjunto):


* properties.csv: Contiene información básica de las características de la propiedad y su información geográfica. Las columnas de este archivo son las siguientes: property_id, business_type, type, bedrooms, bathrooms, latitude, longitude, locality, city & country.


* users.csv: Contiene la información de los propietarios y su relación con la propiedad. Las columnas de este archivo son las siguientes: property_id, user_id, name, last_name & country.


* visits.csv: Contiene la información de los clientes que se han registrado en alguna visita a una propiedad. Las columnas de este archivo son las siguientes: schedule_id, property_id, begin_date, end_date, type_visit & status. Este ultimo campo, puede tomar valor según el estado de la visita y permite verificar si las visitas están agendadas y aún no se realizan (scheduled), canceladas (cancelled) o ya realizadas (Done)


  Por otro lado, Carlos le comenta que existe un servicio para obtener información de las condiciones climáticas de cada país. Se puede consultar este servicio por medio de su API, la cual esta documentada aquí. Se puede consultar la temperatura y clima de una localidad mediante la siguiente consulta.


Ejemplo de Request API Wheather VisualCrossing:

﻿ https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lng}/{start_date}/{end_date}?key={API_KEY}&include=days


Posibles condiciones climáticas retornada por la API:

https://github.com/visualcrossing/WeatherApi/blob/master/lang/en.txt


Nota: Para la utilización de la api se debe proveer el siguiente API KEY. Esta tiene un límite de 1.000 registros al día. Debes registrarte en https://www.visualcrossing.com/sign-up para obtener la KEY.


## Preguntas del desafío:

* ¿Cuántas visitas se realizaron en total?
* ¿Cuál es el promedio de propiedades por propietario?
* ¿Cuál era la temperatura promedio de todas las visitas que realizó en la propiedad del propietario con ID 2?
* ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?
* ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?

## Formato de entrega:


  Para entregar este desafío te solicitamos compartirnos un link de un repositorio en github o gitlab. Este repositorio debe contener un archivo en formato ipynb o py con las respuestas. Debes entregar un archivo requierements.txt con las versiones de las librerías ocupadas.

In [1]:
# carpeta raiz fuentes
folder_data = "data/"

In [2]:
# fuentes
properties = folder_data + "properties.csv"
users = folder_data + "users.csv"
visits = folder_data + "visits.csv"

In [3]:
# carga de imports
import pandas as pd
import seaborn as sns
import requests
import json
import statistics
from datetime import datetime

In [4]:
# cargar fuente de propiedades
pdf_properties = pd.read_csv(properties, sep=',', header=[0], encoding='utf8')
pdf_properties.head(3)

,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.058040,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
1,2,departamento,Rental & Sale,1,1,2,NaN,1,False,4.623068,-74.074030,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia
2,3,departamento,Rental,2,2,2,NaN,0,False,4.723909,-74.042336,Usaquen,Bogotá,Región De Cundinamarca,Colombia


In [5]:
# echo un vistazo a los datos en términos generales
pdf_properties.describe()

,property_id,bedrooms,bathrooms,parking_lots,services,balcony,latitude,longitude
count,80.0000,80.000000,80.000000,80.000000,19.000000,80.000000,80.000000,80.000000
mean,40.5000,2.275000,1.725000,0.650000,5264.526316,0.137500,4.685079,-74.116246
std,23.2379,0.871126,0.655551,0.713345,22941.242048,0.346547,0.062736,0.062496
min,1.0000,1.000000,1.000000,0.000000,0.000000,0.000000,4.586188,-74.245544
25%,20.7500,2.000000,1.000000,0.000000,0.000000,0.000000,4.644076,-74.158101
50%,40.5000,2.000000,2.000000,1.000000,1.000000,0.000000,4.685236,-74.111243
75%,60.2500,3.000000,2.000000,1.000000,3.000000,0.000000,4.722041,-74.057694
max,80.0000,5.000000,4.000000,2.000000,100000.000000,1.000000,4.926684,-74.023470


In [6]:
# echo un vistazo a los datos en términos generales
pdf_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_id    80 non-null     int64  
 1   type_house     80 non-null     object 
 2   business_type  80 non-null     object 
 3   bedrooms       80 non-null     int64  
 4   bathrooms      80 non-null     int64  
 5   parking_lots   80 non-null     int64  
 6   services       19 non-null     float64
 7   balcony        80 non-null     int64  
 8   pool           80 non-null     bool   
 9   latitude       80 non-null     float64
 10  longitude      80 non-null     float64
 11  localidad      80 non-null     object 
 12  city           80 non-null     object 
 13  region         80 non-null     object 
 14  country        80 non-null     object 
dtypes: bool(1), float64(3), int64(5), object(6)
memory usage: 9.0+ KB


In [7]:
# cargo la fuente de usuarios.
pdf_users = pd.read_csv(users, sep=',', header=[0], encoding='utf8')
pdf_users.head(3)

,property_id,user_id,first_name,last_name,address
0,31,1,Josephine,Darakjy,4 B Blue Ridge Blvd
1,34,2,Art,Venere,8 W Cerritos Ave #54
2,48,3,Lenna,Paprocki,639 Main St


In [8]:
pdf_users.describe()

,property_id,user_id
count,80.0000,80.0000
mean,40.5000,40.5000
std,23.2379,23.2379
min,1.0000,1.0000
25%,20.7500,20.7500
50%,40.5000,40.5000
75%,60.2500,60.2500
max,80.0000,80.0000


In [9]:
pdf_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  80 non-null     int64 
 1   user_id      80 non-null     int64 
 2   first_name   80 non-null     object
 3   last_name    80 non-null     object
 4   address      80 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.2+ KB


In [10]:
# cargo la fuente de visitas indicando las columnas tipo date
pdf_visits = pd.read_csv(visits, 
                         sep=',', 
                         header=[0],
                         encoding='utf8', 
                         parse_dates=['begin_date', 'end_date'])
pdf_visits.head(3)

,scheduled_id,property_id,begin_date,end_date,type_visit,status
0,169548,1,2022-01-13 10:00:00-03:00,2022-01-13 12:00:00-03:00,Visit,Cancelled
1,184763,1,2022-01-26 18:00:00-03:00,2022-01-26 20:00:00-03:00,Visit,Cancelled
2,186092,1,2022-01-28 12:00:00-03:00,2022-01-28 14:00:00-03:00,Visit,Cancelled


In [11]:
pdf_visits.describe()

,scheduled_id,property_id
count,425.000000,425.000000
mean,188094.894118,40.087059
std,13820.402675,22.943521
min,160130.000000,1.000000
25%,176671.000000,21.000000
50%,190273.000000,38.000000
75%,200307.000000,61.000000
max,208926.000000,80.000000


In [12]:
pdf_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype                                 
---  ------        --------------  -----                                 
 0   scheduled_id  425 non-null    int64                                 
 1   property_id   425 non-null    int64                                 
 2   begin_date    425 non-null    datetime64[ns, pytz.FixedOffset(-180)]
 3   end_date      425 non-null    datetime64[ns, pytz.FixedOffset(-180)]
 4   type_visit    425 non-null    object                                
 5   status        425 non-null    object                                
dtypes: datetime64[ns, pytz.FixedOffset(-180)](2), int64(2), object(2)
memory usage: 20.0+ KB


In [13]:
# reemplazar el timezone 
# pdf_visits['begin_date'].apply(lambda x: x.replace(tzinfo=None))

### ¿Cuántas visitas se realizaron en total?

In [14]:
# La cantidad de visitas efectivamente realizadas(estado Done) es de 139
pdf_visits['status'].loc[pdf_visits['status'] == 'Done'].count()

139

In [15]:
# un groupby y count sobre la columna status entrega información adicional
# interesante (la cantidad de visitas canceladas)
# si es que la muestra es representativa de la población sería bueno ver las posible causas.
# Y más interesante aún identificar las fechas de más cancelaciones.
pdf_visits.groupby(['status'])['status'].count()

status
Cancelled    286
Done         139
Name: status, dtype: int64

### ¿Cuál es el promedio de propiedades por propietario?

In [16]:
# join entre propiedades y usuarios(propietarios)
pdf_properties_users = pdf_properties.join(pdf_users.set_index('property_id'), on='property_id')

In [17]:
pdf_properties_users.head(3)

,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country,user_id,first_name,last_name,address
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.058040,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia,17,Gladys,Rim,322 New Horizon Blvd
1,2,departamento,Rental & Sale,1,1,2,NaN,1,False,4.623068,-74.074030,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia,18,Yuki,Whobrey,1 State Route 27
2,3,departamento,Rental,2,2,2,NaN,0,False,4.723909,-74.042336,Usaquen,Bogotá,Región De Cundinamarca,Colombia,22,Willard,Kolmetz,618 W Yakima Ave


In [18]:
mean_of_properties_by_owner = pdf_properties_users.groupby(['user_id'])['user_id'].count().mean()
mean_of_properties_by_owner

1.0

In [19]:
# exploro un poco más los datos.
pdf_properties_users.city.unique().tolist()

['Cundinamarca', 'Bogotá']

In [20]:
pdf_properties_users.groupby(['city'])['city'].count()

city
Bogotá          62
Cundinamarca    18
Name: city, dtype: int64

In [21]:
pdf_properties_users.groupby(['business_type'])['business_type'].count()

business_type
Rental           67
Rental & Sale    13
Name: business_type, dtype: int64

In [22]:
pdf_properties_users.groupby(['type_house'])['type_house'].count()

type_house
casa             6
departamento    74
Name: type_house, dtype: int64

### ¿Cuál era la temperatura promedio de todas las visitas que realizó en la propiedad del propietario con ID 2?

In [23]:
# Lo que entiendo de la pregunta es encontrar la temperatura promedio de las visitas efectivamente realizadas
# por el usuario(propietario) ID 2

In [24]:
pdf_visits['property_id'].count()

425

In [25]:
pdf_properties_users['property_id'].count()

80

In [26]:
# join entre propiedades y usuarios(propietarios)
pdf_properties_users_visits = pdf_properties_users.join(pdf_visits.set_index('property_id'), on='property_id')
pdf_properties_users_visits.head(3)

,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,...,country,user_id,first_name,last_name,address,scheduled_id,begin_date,end_date,type_visit,status
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,...,Colombia,17,Gladys,Rim,322 New Horizon Blvd,169548,2022-01-13 10:00:00-03:00,2022-01-13 12:00:00-03:00,Visit,Cancelled
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,...,Colombia,17,Gladys,Rim,322 New Horizon Blvd,184763,2022-01-26 18:00:00-03:00,2022-01-26 20:00:00-03:00,Visit,Cancelled
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,...,Colombia,17,Gladys,Rim,322 New Horizon Blvd,186092,2022-01-28 12:00:00-03:00,2022-01-28 14:00:00-03:00,Visit,Cancelled


In [27]:
pdf_properties_users_visits.shape

(425, 24)

In [28]:
pdf_properties_users_visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 425 entries, 0 to 79
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype                                 
---  ------         --------------  -----                                 
 0   property_id    425 non-null    int64                                 
 1   type_house     425 non-null    object                                
 2   business_type  425 non-null    object                                
 3   bedrooms       425 non-null    int64                                 
 4   bathrooms      425 non-null    int64                                 
 5   parking_lots   425 non-null    int64                                 
 6   services       107 non-null    float64                               
 7   balcony        425 non-null    int64                                 
 8   pool           425 non-null    bool                                  
 9   latitude       425 non-null    float64                            

In [29]:
pdf_properties_users_visits.describe()

,property_id,bedrooms,bathrooms,parking_lots,services,balcony,latitude,longitude,user_id,scheduled_id
count,425.000000,425.000000,425.000000,425.000000,107.000000,425.000000,425.000000,425.000000,425.000000,425.000000
mean,40.087059,2.268235,1.790588,0.701176,6543.252336,0.124706,4.687658,-74.109461,39.232941,188094.894118
std,22.943521,0.886775,0.690847,0.725300,24842.691147,0.330774,0.067154,0.059553,23.651256,13820.402675
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,4.586188,-74.245544,1.000000,160130.000000
25%,21.000000,2.000000,1.000000,0.000000,0.000000,0.000000,4.644231,-74.154625,17.000000,176671.000000
50%,38.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.681658,-74.107834,40.000000,190273.000000
75%,61.000000,3.000000,2.000000,1.000000,3.000000,0.000000,4.723018,-74.055260,59.000000,200307.000000
max,80.000000,5.000000,4.000000,2.000000,100000.000000,1.000000,4.926684,-74.023470,80.000000,208926.000000


In [30]:
pdf_properties_users_visits['begin_date_formatted'] = pdf_properties_users_visits['begin_date'].dt.strftime("%Y-%m-%d")
pdf_properties_users_visits['end_date_formatted'] = pdf_properties_users_visits['end_date'].dt.strftime("%Y-%m-%d")
pdf_properties_users_visits.head(3)

,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,...,first_name,last_name,address,scheduled_id,begin_date,end_date,type_visit,status,begin_date_formatted,end_date_formatted
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,...,Gladys,Rim,322 New Horizon Blvd,169548,2022-01-13 10:00:00-03:00,2022-01-13 12:00:00-03:00,Visit,Cancelled,2022-01-13,2022-01-13
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,...,Gladys,Rim,322 New Horizon Blvd,184763,2022-01-26 18:00:00-03:00,2022-01-26 20:00:00-03:00,Visit,Cancelled,2022-01-26,2022-01-26
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,...,Gladys,Rim,322 New Horizon Blvd,186092,2022-01-28 12:00:00-03:00,2022-01-28 14:00:00-03:00,Visit,Cancelled,2022-01-28,2022-01-28


In [31]:
pdf_visits['begin_date_formatted'] = pdf_visits['begin_date'].dt.strftime("%Y-%m-%d")
pdf_visits['end_date_formatted'] = pdf_visits['end_date'].dt.strftime("%Y-%m-%d")
pdf_visits.head(3)

,scheduled_id,property_id,begin_date,end_date,type_visit,status,begin_date_formatted,end_date_formatted
0,169548,1,2022-01-13 10:00:00-03:00,2022-01-13 12:00:00-03:00,Visit,Cancelled,2022-01-13,2022-01-13
1,184763,1,2022-01-26 18:00:00-03:00,2022-01-26 20:00:00-03:00,Visit,Cancelled,2022-01-26,2022-01-26
2,186092,1,2022-01-28 12:00:00-03:00,2022-01-28 14:00:00-03:00,Visit,Cancelled,2022-01-28,2022-01-28


In [32]:
# seguimos conociendo los datos
pdf_properties_users_visits.groupby(['property_id'])['property_id'].count().sort_values(ascending=False).reset_index(name='count').head(10)

,property_id,count
0,3,10
1,71,10
2,35,10
3,27,10
4,1,9
5,22,9
6,74,9
7,65,9
8,56,9
9,40,9


In [33]:
# buscando tener una mirada de los datos en función del estado de las visitas
pdf_properties_users_visits_subset = pdf_properties_users_visits.loc[:, ['property_id', 'user_id', 'status']]
pdf_properties_users_visits_subset.groupby(['property_id', 'status']).count()

user_id
property_id status            
1           Cancelled        9
2           Cancelled        1
            Done             1
3           Cancelled        8
            Done             2
...                        ...
78          Cancelled        1
            Done             3
79          Cancelled        4
80          Cancelled        6
            Done             3

[136 rows x 1 columns]

In [34]:
API_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"

def api_weather_get_temp(lat, lon, start_date, end_date, API_KEY='VCN9ELN2J86KJZAUNMPQBBB6P'):
    """ Función que realiza el request a la API entregada y obtiene una lista de temperaturas """
    try:
        request = API_URL+lat+","+lon+"/"+start_date+"/"+end_date+"?key="+API_KEY+"&include=days"
        response_api_weather = requests.get(request)
        
        json_formatted = json.dumps(response_api_weather.json(), indent=2)
        #print(json_formatted)
        
        l_temp = []
        len_days = len(response_api_weather.json()["days"])
        
        for i in range(len_days):
            l_temp.append(response_api_weather.json()["days"][i]["temp"])
                
        return l_temp
    
    except requests.ConnectionError as error:
        print(error)

In [35]:
def fahrenheit_to_celsius(f):
    """ Función para convertir de fahrenheit a celsius"""
    return (f - 32) / 1.8

In [36]:
# vamos a suponer que cada visita se considera igual a otra si tiene todas sus columnas
# begin_date_formatted,end_date_formatted,latitude,longitude,property_id iguales
# y por lo tanto serán eliminadas las duplicadas.

def avg_visit_temperature_by_user(user_id, unit="f"):
    """ Calcula la temperatura promedio de todas las visitas realizadas en las propiedades del usuario(propietario) ID 2
            user_id: ID del propietario
            unit: unidad de temperatura en la cual se va a entregar el resultado c para celsius y f para fahrenheit.
    """
    # lista de id de propiedades asociadas al usuario. 
    list_of_properties = pdf_users.loc[pdf_users['user_id'] == user_id][['property_id']]
    list_of_id_properties = list_of_properties['property_id'].tolist()
 
    data_total  = pd.DataFrame()
    dates_partial = pd.DataFrame()
    
    #list_of_id_properties = [3,78]
    l_temp = [] 
    # lista de visitas realizadas (en estado Done) correspondientes 
    # a la propiedad del usuario
    for i in range(len(list_of_id_properties)):
        
        # Supuesto: cada propiedad tiene sólo un duenio o usuario que la representa.
        lat_lon = pdf_properties_users.loc[pdf_properties_users['property_id'] == list_of_id_properties[i]][['latitude','longitude']]
        
        # Como habrá N visitas por cada M propiedad implica que tendremos N rangos de fechas para las visitas efectivamente
        # realizadas (estado Done)
        dates_partial = pdf_visits.loc[(pdf_visits['property_id'] == list_of_id_properties[i]) & (pdf_visits['status']=='Done')][['begin_date_formatted', 'end_date_formatted']]
        # dates_total lo dejo para revisión
        
        dates_partial['latitude'] = lat_lon['latitude'].iloc[0]
        dates_partial['longitude'] = lat_lon['longitude'].iloc[0]
        dates_partial['property_id'] = list_of_id_properties[i]
        
        data_total = pd.concat([data_total, dates_partial], axis=0)
        

    print(data_total)
    # eliminamos tuplas duplicadas
    data_total.drop_duplicates(inplace=True)
    data_total = data_total.reset_index()
    print(data_total)

    for i in data_total.index:
        start_date = data_total['begin_date_formatted'][i]
        end_date = data_total['end_date_formatted'][i]
        lat = str(data_total['latitude'][i])
        lon = str(data_total['longitude'][i])

        # consultamos la api para un rango de fechas
        l_temp += api_weather_get_temp(lat, 
                             lon,
                             start_date,
                             end_date)
        print(l_temp)
    
    if unit == 'c':
        avg_temp = fahrenheit_to_celsius(statistics.mean(l_temp))
    elif unit == 'f':
        avg_temp = statistics.mean(l_temp)

    return avg_temp

In [37]:
user_id = 2
# temperatura en fahrenheit por defecto de la API y se puede setear el parámetro a 'c' para que entregue
# el resultado en celsius
print('Average Temperature = {:.1f} , Visits User(Owner) ID = {}'.format(avg_visit_temperature_by_user(2,'f'), user_id))

    begin_date_formatted end_date_formatted  latitude  longitude  property_id
178           2022-01-28         2022-01-29  4.618363   -74.0735           34
179           2022-01-28         2022-01-28  4.618363   -74.0735           34
   index begin_date_formatted end_date_formatted  latitude  longitude  \
0    178           2022-01-28         2022-01-29  4.618363   -74.0735   
1    179           2022-01-28         2022-01-28  4.618363   -74.0735   

   property_id  
0           34  
1           34  
[58.3, 56.1]
[58.3, 56.1, 58.3]
Average Temperature = 57.6 , Visits User(Owner) ID = 2


### ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?

In [38]:
pdf_properties_users_visits_subset_condition = pdf_properties_users_visits.loc[:, ['property_id', 'user_id', 'status', 'latitude', 'longitude', 'begin_date_formatted', 'end_date_formatted', 'localidad']]
pdf_properties_users_visits_subset_condition.drop_duplicates(inplace=True)
pdf_properties_users_visits_subset_condition.reset_index().head()

,index,property_id,user_id,status,latitude,longitude,begin_date_formatted,end_date_formatted,localidad
0,0,1,17,Cancelled,4.870956,-74.05804,2022-01-13,2022-01-13,Chí­A
1,0,1,17,Cancelled,4.870956,-74.05804,2022-01-26,2022-01-26,Chí­A
2,0,1,17,Cancelled,4.870956,-74.05804,2022-01-28,2022-01-28,Chí­A
3,0,1,17,Cancelled,4.870956,-74.05804,2022-01-23,2022-01-23,Chí­A
4,0,1,17,Cancelled,4.870956,-74.05804,2022-01-11,2022-01-11,Chí­A


In [39]:
pdf_properties_users_visits_subset_condition.describe()

,property_id,user_id,latitude,longitude
count,342.000000,342.000000,342.000000,342.000000
mean,38.590643,38.078947,4.684970,-74.108409
std,22.249761,22.937175,0.061805,0.058708
min,1.000000,1.000000,4.586188,-74.245544
25%,19.250000,17.000000,4.644231,-74.154280
50%,37.000000,38.000000,4.681658,-74.107834
75%,57.750000,58.000000,4.721424,-74.055260
max,80.000000,80.000000,4.926684,-74.023470


In [40]:
# Nota:
# como cada visita está y debe estar asociada a una ubicación de una propiedad
# sería interesante revisar este promedio para las propiedades que han recibido
# mayores cancelaciones de la visita y eventualmente apreciar cierto comportamiento
# asociado a la condición climática.

In [41]:
API_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"

def api_weather_get_temp_by_condition(c, lat, lon, start_date, end_date, API_KEY='VCN9ELN2J86KJZAUNMPQBBB6P'):
    """ Función que realiza el request a la API entregada y obtiene una lista de temperaturas """
    try:
        request = API_URL+lat+","+lon+"/"+start_date+"/"+end_date+"?key="+API_KEY+"&include=days"
        response_api_weather = requests.get(request)
        
        json_formatted = json.dumps(response_api_weather.json(), indent=2)
        #print(json_formatted)
        
        l_temp = []
        l_condition = []
        len_days = len(response_api_weather.json()["days"])
        
        for i in range(len_days):
            l_condition = response_api_weather.json()["days"][i]["conditions"]
            if c in l_condition:
                l_temp.append(response_api_weather.json()["days"][i]["temp"])
                
        return l_temp
    
    except requests.ConnectionError as error:
        print(error)

In [47]:
# vamos a suponer que cada visita se considera igual a otra si tiene todas sus columnas
# begin_date_formatted,end_date_formatted,latitude,longitude,property_id iguales
# y por lo tanto serán eliminadas las duplicadas.

# la unica forma de saber cuales fueron los dias con lluvia es consultar a la API por cada visita.
def avg_visit_temperature_by_condition(c, status='Done', unit="f"):
    """ Calcula la temperatura promedio de las visitas efectivamente realizadas por localidad
            c: condicion del tiempo indicada en la lista entregada
            status: por defecto es Done
            unit: unidad de temperatura en la cual se va a entregar el resultado c para celsius y f para fahrenheit.
    """
    
    pdf_condition = pdf_properties_users_visits_subset_condition.loc[pdf_properties_users_visits_subset_condition['status'] == status].reset_index()

    # trabajaremos con un subconjunto de datos
    l_temp = [] 
   
    for i in pdf_condition.index:
        start_date = pdf_condition['begin_date_formatted'][i]
        end_date = pdf_condition['end_date_formatted'][i]
        lat = str(pdf_condition['latitude'][i])
        lon = str(pdf_condition['longitude'][i])

        # consultamos la api para un rango de fechas
        l_temp += api_weather_get_temp_by_condition(c, lat, 
                             lon,
                             start_date,
                             end_date)
        print(l_temp)
    
    if unit == 'c':
        avg_temp = fahrenheit_to_celsius(statistics.mean(l_temp))
    elif unit == 'f':
        avg_temp = statistics.mean(l_temp)

    return avg_temp

In [48]:
condition = 'Rain'
status = 'Done'
result_by_condition = avg_visit_temperature_by_condition(condition, status, unit='f')

print(result_by_condition)

[58.2]
[58.2, 56.1]
[58.2, 56.1, 56.0]
[58.2, 56.1, 56.0, 55.5]
[58.2, 56.1, 56.0, 55.5]
[58.2, 56.1, 56.0, 55.5, 56.0]
[58.2, 56.1, 56.0, 55.5, 56.0]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7, 57.2]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7, 57.2]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7, 57.2, 56.7]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7, 57.2, 56.7, 57.6]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7, 57.2, 56.7, 57.6, 58.2]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7, 57.2, 56.7, 57.6, 58.2, 59.3]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59.1, 57.5, 57.7, 57.2, 56.7, 57.6, 58.2, 59.3, 57.7]
[58.2, 56.1, 56.0, 55.5, 56.0, 58.0, 57.8, 59

### ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?

In [49]:
# primero voy a filtrar por la localidad de suba y asi obtener las visitas asociadas a las propiedades de esa localidad
# además filtraré por las visitas efectivamente realizadas (Done)
pdf_visits_suba = pdf_properties_users_visits_subset_condition.loc[(pdf_properties_users_visits_subset_condition['localidad'] == 'Suba') 
                                                                   & (pdf_properties_users_visits_subset_condition['status'] == 'Done')].reset_index()
pdf_visits_suba.head()

,index,property_id,user_id,status,latitude,longitude,begin_date_formatted,end_date_formatted,localidad
0,9,10,28,Done,4.723203,-74.053140,2022-01-04,2022-01-04,Suba
1,18,19,35,Done,4.767047,-74.048990,2022-01-14,2022-01-14,Suba
2,18,19,35,Done,4.767047,-74.048990,2022-01-03,2022-01-03,Suba
3,27,28,19,Done,4.753546,-74.093185,2022-01-17,2022-01-17,Suba
4,32,33,47,Done,4.713548,-74.088590,2022-02-07,2022-02-07,Suba


In [50]:
pdf_visits_suba.groupby(['status'])['status'].count()

status
Done    30
Name: status, dtype: int64

In [51]:
# vamos a suponer que cada visita se considera igual a otra si tiene todas sus columnas
# begin_date_formatted,end_date_formatted,latitude,longitude,property_id iguales
# y por lo tanto serán eliminadas las duplicadas.

# la unica forma de saber cuales fueron los dias con lluvia es consultar a la API por cada visita.

def avg_visit_temperature_by_localidad(localidad, status='Done', unit="f"):
    """ Calcula la temperatura promedio de las visitas efectivamente realizadas por localidad
            localidad: pueder ser cualquier localidad dentro del dataset
            status: por defecto es Done
            unit: unidad de temperatura en la cual se va a entregar el resultado c para celsius y f para fahrenheit.
    """
    
    pdf_visits_localidad = pdf_properties_users_visits_subset_condition.loc[(pdf_properties_users_visits_subset_condition['localidad'] == localidad) 
                                                                   & (pdf_properties_users_visits_subset_condition['status'] == status)].reset_index()

    # trabajaremos con un subconjunto de datos
    l_temp = [] 
   
    for i in pdf_visits_localidad.index:
        start_date = pdf_visits_localidad['begin_date_formatted'][i]
        end_date = pdf_visits_localidad['end_date_formatted'][i]
        lat = str(pdf_visits_localidad['latitude'][i])
        lon = str(pdf_visits_localidad['longitude'][i])

        # consultamos la api para un rango de fechas
        l_temp += api_weather_get_temp(lat, 
                             lon,
                             start_date,
                             end_date)
        print(l_temp)
    
    if unit == 'c':
        avg_temp = fahrenheit_to_celsius(statistics.mean(l_temp))
    elif unit == 'f':
        avg_temp = statistics.mean(l_temp)

    return avg_temp

In [52]:
localidad = 'Suba'
result_by_localidad = avg_visit_temperature_by_localidad(localidad, status='Done', unit='f')
print('Average Temperature = {:.1f} , Localidad = {}'.format(result_by_localidad, 'Suba'))

[56.7]
[56.7, 56.2]
[56.7, 56.2, 57.7]
[56.7, 56.2, 57.7, 55.9]
[56.7, 56.2, 57.7, 55.9, 57.3]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3, 59.6]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3, 59.6, 59.5]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3, 59.6, 59.5, 56.0]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3, 59.6, 59.5, 56.0, 58.7]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3, 59.6, 59.5, 56.0, 58.7, 59.5]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3, 59.6, 59.5, 56.0, 58.7, 59.5, 56.5]
[56.7, 56.2, 57.7, 55.9, 57.3, 55.1, 59.6, 59.5, 55.5, 55.5, 55.3

# Conclusiones
* Aterrizando un poco las preguntas se pueden hacer mejores análisis de la data y evitar perder tiempo en ello.
* Hay espacio para modularizar, optimizar, documentar en mayor medida y utilizando herramientas más completas que sólo el doc por defecto de python por ejemplo.
* Esta función pudo ser una sola api_weather_get_temp para todos y dado que estaba demasiado corto en consultas no quise mover nada al final.